In [ ]:
import flopy as fp
import pyemu
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, shutil, glob, sys
import json

In [ ]:
pyemu.__version__

In [ ]:
sim_ws = '../neversink_mf6/'
template_ws = '../run_data'
basename = 'neversink_ies'

#### kill the `original` folder

In [ ]:
if os.path.exists(os.path.join(sim_ws,'original')):
    shutil.rmtree(os.path.join(sim_ws,'original'))

In [ ]:
run_MF6 = True # option to run MF6 to generate output but not needed if already been run in sim_ws
cdir = os.getcwd()


# optionally run MF6 to generate model output
if run_MF6:
    os.chdir(sim_ws)
    os.system('mf6')
    os.chdir(cdir)

### create land surface observations we will need at the end

In [ ]:
irch_file = f'{sim_ws}/irch.dat'
id3_file = f'{sim_ws}/idomain_003.dat'
top_file = f'{sim_ws}/top.dat'

In [ ]:
top = np.loadtxt(top_file)
top[top<-8000] = np.nan
plt.imshow(top)

plt.colorbar()

In [ ]:
id3  = np.loadtxt(id3_file, dtype=int)
plt.imshow(id3)

In [ ]:
irch = np.loadtxt(irch_file, dtype=int)
irch -= 1
plt.imshow(irch)
plt.colorbar()

In [ ]:
# set frequency for land surface observations lateralls, in model cells
lsobs_every_n_cells = 50 

In [ ]:
# make a grid of cells spaced at the spacing suggested above
nrow, ncol = id3.shape
j = list(range(ncol))[0:ncol:lsobs_every_n_cells]
i = list(range(nrow))[0:nrow:lsobs_every_n_cells]
J,I = np.meshgrid(j,i)
points = list(zip(I.ravel(),J.ravel()))

In [ ]:
# now keep only those that are in active cells (using ibound of layer4 as the basis) and drop a few others

In [ ]:
keep_points = [(irch[i,j],i,j) for i,j in points if id3[i,j]==1]
drop_points = [(0, 150, 50),(3, 150, 100),(3, 100, 50)]
keep_points = [i for i in keep_points if i not in drop_points]

### make list of indices

In [ ]:
with open(os.path.join(sim_ws,'land_surf_obs-indices.csv'), 'w') as ofp:
    ofp.write('k,i,j,obsname\n')
    [ofp.write('{0},{1},{2},land_surf_obs_{1}_{2}\n'.format(*i)) for i in keep_points]

### make an observations file

In [ ]:
with open(os.path.join(sim_ws,'land_surf_obs-observations.csv'), 'w') as ofp:
    ofp.write('obsname,obsval\n')
    [ofp.write('land_surf_obs_{1}_{2},{3}\n'.format(*i, top[i[1],i[2]])) for i in keep_points]

### load up the simulation

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws=sim_ws)

In [ ]:
m = sim.get_model()

In [ ]:
# manually create a spatial reference object from the grid.json metadata
grid_data = json.load(open(os.path.join(sim_ws,'neversink_grid.json')))

In [ ]:
sr_model = pyemu.helpers.SpatialReference(delr=grid_data['delr'],
                                           delc=grid_data['delc'],
                                           rotation= grid_data['angrot'],
                                           epsg = grid_data['epsg'],
                                           xul = grid_data['xul'],
                                           yul = grid_data['yul'],
                                           units='meters',
                                           lenuni=grid_data['lenuni'])

In [ ]:
# create the PstFrom object 
pf = pyemu.utils.PstFrom(original_d=sim_ws, new_d=template_ws,
                 remove_existing=True,
                 longnames=True, spatial_reference=sr_model,
                 zero_based=False)

## we will parameterize....
- pilot points for k, k33, r
- zones for l, k33, r
- constant for R
- sfr conductance by reach
- well pumping 
- CHDs

In [ ]:
# parameterize list-directed well and chd packages
list_tags = {'wel_':[.8,1.2], 'chd_':[.8,1.2]}

In [ ]:
for tag,bnd in list_tags.items():
    lb,ub = bnd
    filename = os.path.basename(glob.glob(os.path.join(template_ws, '*{}*'.format(tag)))[0])
    pf.add_parameters(filenames=filename, par_type = 'grid',
                     upper_bound=ub, lower_bound=lb, par_name_base=tag,
                     index_cols=[0,1,2], use_cols=[3],pargp=tag[:-1],alt_inst_str='',
                     comment_char='#')

In [ ]:
k_ub = 152
# set up pilot points
pp_tags = {'k':[.01,10.,k_ub], 'k33':[.01,10.,k_ub/10]}

In [ ]:
idm = m.dis.idomain.array

In [ ]:
plt.imshow(idm[2])
plt.colorbar()

In [ ]:
idm[idm==-1]=0

In [ ]:
for i in range(4):
    plt.figure()
    plt.imshow(idm[i])
    plt.colorbar()

#### before setting up K, need to edit the zone files to only have nonzero values in active cells

In [ ]:
kzonefile = '../k_processing/padded_L{}_K_Zone_50mGrid.dat'
zonearrs = {}
for i in range(m.dis.nlay.data):
    kz = np.loadtxt(kzonefile.format(i)).astype(int)
    kz[idm[i] != 1] = 0 # CHANGED FROM 0 TO 'i' ON 11/23/2020
    zonearrs[i] = kz


In [ ]:
for i in range(4):
    plt.figure()
    plt.imshow(zonearrs[i])
    plt.colorbar()

In [ ]:
[np.unique(kz) for _,kz in zonearrs.items()]

In [ ]:
tag='k33'

In [ ]:
arrfiles = [f for f in os.listdir(template_ws) if f.startswith(tag)]# & ('k33' not in f)]
arrfiles

In [ ]:
[int(re.findall('\d+',i.replace('k33',''))[-1]) for i in arrfiles]

In [ ]:
## set up for K
for tag,bnd in pp_tags.items():
    lb, ub, ultub = bnd
    if tag == 'k':
        arrfiles = sorted([f for f in os.listdir(template_ws) if f.startswith(tag) & ('k33' not in f)])
    else:
        arrfiles = sorted([f for f in os.listdir(template_ws) if f.startswith(tag)])
    
    for arr_file in arrfiles:        
        currlay = int(re.findall('\d+',arr_file.replace('k33',''))[-1])
        
        # pilot points
        # set pilot point spacing:
        if currlay in [1,2]:
            pp_space = 5           
        else: 
            pp_space = 20
        v = pyemu.utils.geostats.ExpVario(a=sr_model.delr[0]*3*pp_space,contribution=1.0)
        gs = pyemu.utils.geostats.GeoStruct(variograms=v,nugget=0.0, transform='log')

        print('pps for layer {} --- filename: {}: idomain_sum: {}'.format(currlay, arr_file, idm[currlay].sum()))
        pf.add_parameters(filenames=arr_file, par_type='pilotpoints', pp_space=pp_space,
                         upper_bound=ub, lower_bound=lb, geostruct=gs,
                         par_name_base='{}_pp'.format(tag),alt_inst_str='',
                         zone_array=idm[currlay], pargp='pp_{}'.format(tag),
                         ult_ubound=ultub)
        # zones
        print('zones for layer {} --- filename: {}: idomain_sum: {}'.format(currlay, arr_file, idm[currlay].sum()))
        pf.add_parameters(filenames=arr_file, par_type='zone',alt_inst_str='',
                         zone_array = zonearrs[currlay],lower_bound=lb,upper_bound=ub,
                         pargp='zn_{}'.format(tag), par_name_base='{}_{}'.format(tag,currlay),
                          ult_ubound=ultub)
        

In [ ]:
# recharge as special case because no idomain for R
rtags= {'rch':[0.8,1.2, np.max(m.rch.recharge.array)*1.2]}

In [ ]:
for tag,bnd in rtags.items():
    lb, ub, ultub = bnd
    if tag == 'k':
        arrfiles = sorted([f for f in os.listdir(template_ws) if f.startswith(tag) & ('k33' not in f)])
    else:
        arrfiles = sorted([f for f in os.listdir(template_ws) if f.startswith(tag)])
    
    for arr_file in arrfiles:
        # pilot points
        pf.add_parameters(filenames=arr_file, par_type='pilotpoints', pp_space=pp_space,
                         upper_bound=ub, lower_bound=lb, geostruct=gs,
                         par_name_base='{}_pp'.format(tag),
                          zone_array=idm[3],alt_inst_str='',
                         pargp='pp_{}'.format(tag),
                          ult_ubound=ultub)
        # constant
        pf.add_parameters(filenames=arr_file, par_type='constant',
                 upper_bound=ub-0.1, lower_bound=lb+0.1, 
                 par_name_base='{}_const'.format(tag),
                 zone_array=idm[3],alt_inst_str='',
                 pargp='pp_{}'.format(tag),
                           ult_ubound=ultub)


### Make a TPL file for SFR and add it to the `pst` object

In [ ]:
sfrfilename = 'neversink_packagedata.dat'

print('working on {}'.format(sfrfilename))
# read in and strip and split the input lines
insfr = [line.strip().split() for line in open(os.path.join(template_ws,sfrfilename), 'r').readlines() if '#' not in line]
headerlines = [line.strip() for line in open(os.path.join(template_ws,sfrfilename), 'r').readlines() if '#' in line]

# set up the template line strings by segment
tpl_char = ['~ sfrk_{} ~'.format(line[-1]) for line in insfr]

# stick the tpl text in the K column. NB -> gotta count from the end because of 
# the possibility of NONE or i,j,k as indexing
for line,tpl in zip(insfr,tpl_char):
    line[-6] = tpl

# revert back to a space delimited file
insfr = [' '.join(line) for line in insfr]

# write out the TPL file
with open(os.path.join(template_ws,'{}.tpl'.format(sfrfilename)), 'w') as ofp:
    ofp.write('ptf ~\n')
    [ofp.write('{}\n'.format(line)) for line in headerlines]
    [ofp.write('{}\n'.format(line)) for line in insfr]

In [ ]:
pst = pf.build_pst('{}.pst'.format(basename))

In [ ]:
pst.add_parameters(os.path.join(template_ws,'{}.tpl'.format(sfrfilename)), pst_path='.')

In [ ]:
pst.parameter_data.loc[pst.parameter_data.parnme.str.startswith('sfr'),'pargp'] = 'sfrk'

## Add in the observations - updating weights and values to come later

In [ ]:
shutil.copy2('../scripts/get_observations.py',os.path.join(template_ws,'get_observations.py'))

In [ ]:
os.system('python {}'.format(os.path.join(template_ws,'get_observations.py')))

In [ ]:
pst.add_observations(os.path.join(template_ws,'obs_mf6.dat.ins'), pst_path='.')

In [ ]:
pst.observation_data

In [ ]:
pst.write(os.path.join(template_ws,'temp.pst'))